## 🧬 Workflow

1. Input Data
- Variant-level dataset (matched_variants_any_plp.tsv)

Total variants loaded: 124365

Come from VAT table filtering with vid.

<span style="color:red">clinvar_classification includes 'benign, uncertain significance', need to delete?</span>

- Gene–transcript mapping file (gene_transcript_mapping.csv)

299 genes(296 list) matched with mane transcripts. 

One gene might connect with over one transcripts.

2. Preprocessing & Mapping
- Clean gene_symbol (only in 296 list) and transcript columns.
- Derive mane_transcript from transcript IDs.
- Inner join with mapping file to retain variants correctly mapped to MANE transcripts.

3. ClinVar Conflict Flagging
- Normalize clinvar_classification.
- Define sets: benign (benign, likely benign), pathogenic (pathogenic, likely pathogenic).
- Assign conflict_flag = 1 if a variant annotation contains both benign and pathogenic.

4. Count conflict variants per gene.

5. Allele Frequency Flagging
- Convert gvs_all_af to numeric.
- Flag variants as af_flag = 1 if AF > 1%.
- Save high AF variants separately to output_high_af_variant.tsv.
- Save cleaned, flagged, non-rare variant dataset (analyzed_variants_any_plp.tsv).

<span style="color:red">Retain only rare variants (AF ≤ 1%) for downstream gene-level analysis?</span>


6. Gene-level Analysis
- Select allele count (AC) and allele number (AN) columns.
- Convert to numeric.
- For each population (all, afr, amr, eas, eur, mid, oth, sas):

exome_<pop>_ac = sum of ACs across all rare variants in the gene.

exome_<pop>_an = mean of ANs across all rare variants in the gene.

exome_<pop>_cf_gene = composite frequency per gene = 2 * sum_ac / mean_an.

Also calculate per-gene conflict_count for output.

7. Separate XL Genes
- Use a predefined gene set ('IDS','EDA','SLC16A2','F8','F9','IL2RG','MAGED2','NR0B1','AVPR2',
        'IGSF1','SOX3','BTK','CD40LG','IKBKG','WAS','CYBB','FOXP3','SH2D1A',
        'XIAP','MSN','TAZ','ATP7A','OTC','PDHA1','SLC35A2').
- Split into XL genes vs other (autosomal and non-XL).

8. Outputs
- *_autosome.tsv → autosomal/other genes.
- *_XLgenes.tsv → XL gene set.

Both outputs include:
gene_symbol
conflict_count

For each population: `exome_<pop>_ac`, `exome_<pop>_an`, `exome_<pop>_cf_gene`.


## Conclusion
1. "analyzed_variants_any_plp.tsv" （non-high AF variants for downstream analysis）
2. "output_high_af_variant.tsv" （high AF variants）
3. "analyzed_genes_any_plp_autosome.tsv" （gene level result without XL genes）
4. "analyzed_genes_any_plp_XLgenes.tsv" （gene level result for XL genes）

In [8]:
import pandas as pd
import numpy as np

def filter_variants_by_mapping():
    # === File paths ===
    variant_file = r"D:\DUKE\LAB\LOEUF\hail296\withaf\matched_variants_any_plp.tsv"
    mapping_file = r"D:\DUKE\LAB\LOEUF\hail296\gene_transcript_mapping.csv"
    output_file_variant = r"D:\DUKE\LAB\LOEUF\hail296\withaf\0908\analyzed_variants_any_plp.tsv"
    output_file_gene = r"D:\DUKE\LAB\LOEUF\hail296\withaf\0908\analyzed_genes_any_plp.tsv"

    # === Step 1: Load variant file ===
    print("📥 Loading variant file...")
    df_variants = pd.read_csv(variant_file, sep="\t", low_memory=False)
    print(f"✅ Total variants loaded: {len(df_variants)}")

    # Clean gene_symbol and transcript columns
    df_variants["gene_symbol"] = df_variants["gene_symbol"].astype(str).str.strip()
    df_variants["transcript"] = df_variants["transcript"].astype(str).str.strip()
    df_variants["mane_transcript"] = df_variants["transcript"].str.split(".").str[0]

    # === Step 2: Load mapping file ===
    print("📥 Loading mapping file...")
    mapping_df = pd.read_csv(mapping_file)
    mapping_df["gene_symbol"] = mapping_df["gene_symbol"].astype(str).str.strip()
    mapping_df["mane_transcript"] = mapping_df["mane_transcript"].astype(str).str.strip()
    print(f"✅ Mapping file genes: {mapping_df['gene_symbol'].nunique()}")

    # === Step 3: Filter variants by mapping (inner merge on gene + transcript) ===
    print("🔍 Filtering variants by gene symbol and MANE transcript mapping...")
    filtered = df_variants.merge(
        mapping_df[["gene_symbol", "mane_transcript"]],
        on=["gene_symbol", "mane_transcript"],
        how="inner"
    )

    print(f"✅ Variants after mapping filter: {len(filtered)}")
    print(f"   → Unique VIDs: {filtered['vid'].nunique()}")
    print(f"   → Unique gene_symbols: {filtered['gene_symbol'].nunique()}")

    # === Step 4: Add conflict_flag based on clinvar_classification ===
    print("🚩 Adding conflict flags based on ClinVar classifications...")
    filtered["clinvar_classification"] = filtered["clinvar_classification"].astype(str).str.strip().str.lower()
    
    print(f"Unique clinvar_classification in the variant list shown:")
    print(filtered["clinvar_classification"].unique())

    # Define classification sets
    benign_set = {"benign", "likely benign"}
    pathogenic_set = {"pathogenic", "likely pathogenic"}

    def check_conflict(val):
        if pd.isna(val) or val == 'nan' or val == '':
            return 0  # No conflict for missing values
        classes = set([x.strip().lower() for x in str(val).split(",")])
        has_benign = not benign_set.isdisjoint(classes)
        has_pathogenic = not pathogenic_set.isdisjoint(classes)
        return 1 if (has_benign and has_pathogenic) else 0

    # Add binary conflict flag (0/1)
    filtered["conflict_flag"] = filtered["clinvar_classification"].apply(check_conflict)
    
    # Count conflicts per gene
    conflict_counts = filtered[filtered["conflict_flag"] == 1]["gene_symbol"].value_counts()
    print(f"✅ Genes with conflicts: {len(conflict_counts)}")
    if len(conflict_counts) > 0:
        print("Top genes with conflict variants:")
        print(conflict_counts.head(10))

    # === Step 5: Add allele frequency flag (>1% = 1, <=1% = 0) ===
    print("📊 Adding allele frequency flags...")
    
    filtered["gvs_all_af"] = pd.to_numeric(filtered["gvs_all_af"], errors="coerce").fillna(0)
    filtered['af_flag'] = filtered['gvs_all_af'].apply(lambda x: 1 if x > 0.01 else 0)
    
    high_af_count = (filtered['af_flag'] == 1).sum()
    print(f"✅ Variants with >1% allele frequency: {high_af_count}")
    # === Step 5.1: Save high AF variants separately ===
    output_high_af_variant = r"D:\DUKE\LAB\LOEUF\hail296\withaf\0908\output_high_af_variant.tsv"
    high_af_variants = filtered[filtered['af_flag'] == 1]
    high_af_variants.to_csv(output_high_af_variant, sep="\t", index=False)
    print(f"💾 High AF variants saved to: {output_high_af_variant}")

    # === Step 6: Save cleaned and flagged variant list ===
    print("💾 Saving cleaned and flagged variant list High ...")
    # keep only AF ≤ 1% variants（rare variants）
    df_gene_af_rare = filtered[filtered['af_flag'] == 0].copy()
    print(f"✅ Variants kept for AF/AN calculation (AF ≤ 1%): {len(df_gene_af_rare)}")

    df_gene_af_rare.to_csv(output_file_variant, sep="\t", index=False)
    print(f"✅ Variant file saved to: {output_file_variant}")

    # === Step 7: Calculate gene-level allele frequencies ===
    print("🧬 Calculating gene-level allele frequencies...")

    # Select relevant columns for gene-level analysis
    relevant_columns = [
        'vid', 'transcript', 'alt_allele',
        'gvs_all_ac', 'gvs_all_an',
        'gvs_afr_ac', 'gvs_afr_an',
        'gvs_amr_ac', 'gvs_amr_an',
        'gvs_eas_ac', 'gvs_eas_an',
        'gvs_eur_ac', 'gvs_eur_an',
        'gvs_mid_ac', 'gvs_mid_an',
        'gvs_oth_ac', 'gvs_oth_an',
        'gvs_sas_ac', 'gvs_sas_an',
        'gene_symbol', 'consequence', 'clinvar_classification',
        'conflict_flag', 'af_flag'
    ]

    df_gene_af = df_gene_af_rare[relevant_columns].copy()

    # === Step 8: Convert AC/AN columns to numeric ===
    ac_an_cols = [c for c in df_gene_af.columns if c.endswith('_ac') or c.endswith('_an')]
    df_gene_af[ac_an_cols] = df_gene_af[ac_an_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

    # === Step 9: Define populations ===
    pops = ['all', 'afr', 'amr', 'eas', 'eur', 'mid', 'oth', 'sas']

    # === Step 10: Gene-level aggregation (sum AC, mean AN, conflict count) ===
    grouped = df_gene_af.groupby(['gene_symbol'], as_index=False).agg(
        conflict_count=("conflict_flag", "sum"),
        **{
            f"exome_{p}_ac": (f"gvs_{p}_ac", "sum")
            for p in pops
        },
        **{
            f"exome_{p}_an": (f"gvs_{p}_an", "mean")
            for p in pops
        }
    )

    # === Step 11: Calculate CF-gene (per population) ===
    for p in pops:
        grouped[f"exome_{p}_cf_gene"] = np.where(
            grouped[f"exome_{p}_an"] > 0,
            2 * grouped[f"exome_{p}_ac"] / grouped[f"exome_{p}_an"],
            np.nan
        )

    # === Step 12: Prepare output columns (ordered) ===
    output_cols = (
        ["gene_symbol", "conflict_count"]  # add conflict_count
        + sum(
            [[f"exome_{p}_ac", f"exome_{p}_an", f"exome_{p}_cf_gene"] for p in pops],
            []
        )
    )


    # === Step 13: Separate XL genes ===
    XLgenes = set([
        'IDS','EDA','SLC16A2','F8','F9','IL2RG','MAGED2','NR0B1','AVPR2',
        'IGSF1','SOX3','BTK','CD40LG','IKBKG','WAS','CYBB','FOXP3','SH2D1A',
        'XIAP','MSN','TAZ','ATP7A','OTC','PDHA1','SLC35A2'
    ])

    df_xl = grouped[grouped["gene_symbol"].isin(XLgenes)]
    df_other = grouped[~grouped["gene_symbol"].isin(XLgenes)]

    # === Step 14: Save results ===
    print("💾 Saving gene-level analysis (exome format)...")
    df_other[output_cols].to_csv(output_file_gene.replace(".tsv", "_autosome.tsv"), sep="\t", index=False)
    print(f"✅ Autosomal/other genes saved to: {output_file_gene.replace('.tsv', '_autosome.tsv')}")

    df_xl[output_cols].to_csv(output_file_gene.replace(".tsv", "_XLgenes.tsv"), sep="\t", index=False)
    print(f"✅ XL genes saved to: {output_file_gene.replace('.tsv', '_XLgenes.tsv')}")

if __name__ == "__main__":
    filter_variants_by_mapping()





📥 Loading variant file...
✅ Total variants loaded: 124365
📥 Loading mapping file...
✅ Mapping file genes: 299
🔍 Filtering variants by gene symbol and MANE transcript mapping...
✅ Variants after mapping filter: 10387
   → Unique VIDs: 10030
   → Unique gene_symbols: 262
🚩 Adding conflict flags based on ClinVar classifications...
Unique clinvar_classification in the variant list shown:
['likely pathogenic, not provided' 'pathogenic'
 'likely pathogenic, pathogenic' 'nan' 'likely pathogenic'
 'uncertain significance, pathogenic' 'uncertain significance'
 'uncertain significance, not provided'
 'uncertain significance, likely pathogenic'
 'uncertain significance, likely pathogenic, pathogenic'
 'likely benign, uncertain significance, pathogenic'
 'uncertain significance, likely pathogenic, pathogenic, not provided'
 'likely pathogenic, pathogenic, risk factor'
 'uncertain significance, pathogenic, risk factor'
 'benign, likely benign, uncertain significance, likely pathogenic'
 'likely pat